In [ ]:
# Following https://github.com/huggingface/notebooks/blob/master/examples/translation.ipynb

In [1]:
!pip install wandb -q

     |████████████████████████████████| 1.7 MB 2.3 MB/s 
     |████████████████████████████████| 139 kB 40.8 MB/s 
     |████████████████████████████████| 97 kB 5.8 MB/s 
     |████████████████████████████████| 180 kB 42.9 MB/s 
     |████████████████████████████████| 63 kB 1.5 MB/s 


In [55]:
# Flexible integration for any Python script
import wandb

# 1. Start a W&B run
wandb.init(project='text-to-sparql', entity='shahriar', )

# 2. Save model inputs and hyperparameters
config = wandb.config
config.learning_rate = 0.01

In [3]:
from google.colab import drive
drive.mount('drive')

Mounted at drive


In [2]:
!unzip lc-quad-wikidata-2021-10-01.zip

Archive:  lc-quad-wikidata-2021-10-01.zip
   creating: lc-quad-wikidata-2021-10-01/
 extracting: lc-quad-wikidata-2021-10-01/dataset_dict.json  
   creating: lc-quad-wikidata-2021-10-01/test/
  inflating: lc-quad-wikidata-2021-10-01/test/state.json  
  inflating: lc-quad-wikidata-2021-10-01/test/dataset_info.json  
  inflating: lc-quad-wikidata-2021-10-01/test/dataset.arrow  
   creating: lc-quad-wikidata-2021-10-01/train/
  inflating: lc-quad-wikidata-2021-10-01/train/state.json  
  inflating: lc-quad-wikidata-2021-10-01/train/dataset_info.json  
  inflating: lc-quad-wikidata-2021-10-01/train/dataset.arrow  


In [5]:
!nvidia-smi

Fri Oct  1 22:18:49 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.74       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   66C    P8    33W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [9]:
import datetime
datetime.datetime.now().strftime("%Y-%m-%d")

'2021-10-01'

In [21]:
model_checkpoint = 't5-small'
fp16 = True
todaydate = datetime.datetime.now().strftime("%Y-%m-%d")
gdir = f'drive/My Drive/Colab Notebooks/{todaydate}/'
model_name=f'sparql-translator-{todaydate}-{model_checkpoint}' + '-fp16' if fp16 else ''
model_path='./models/'+model_name
ds_path= f'lc-quad-wikidata-{todaydate}'

In [22]:
model_name

'sparql-translator-2021-10-01-t5-small-fp16'

In [23]:
!mkdir models

In [25]:
!pip install datasets transformers sacrebleu==1.5.1 -qqq

     |████████████████████████████████| 270 kB 3.2 MB/s 
     |████████████████████████████████| 2.9 MB 40.0 MB/s 
     |████████████████████████████████| 54 kB 2.6 MB/s 
     |████████████████████████████████| 123 kB 42.0 MB/s 
     |████████████████████████████████| 52 kB 1.3 MB/s 
     |████████████████████████████████| 1.3 MB 28.4 MB/s 
     |████████████████████████████████| 243 kB 47.4 MB/s 
     |████████████████████████████████| 895 kB 38.3 MB/s 
     |████████████████████████████████| 3.3 MB 31.8 MB/s 
     |████████████████████████████████| 636 kB 39.8 MB/s 
     |████████████████████████████████| 142 kB 42.4 MB/s 
     |████████████████████████████████| 294 kB 45.2 MB/s 


In [26]:
from datasets import load_dataset, load_metric, Dataset, load_from_disk


In [27]:
raw_datasets = load_from_disk(ds_path)

In [28]:
!pip install tqdm==4.49.0

     |████████████████████████████████| 69 kB 2.5 MB/s 
  Attempting uninstall: tqdm
    Found existing installation: tqdm 4.62.3
    Uninstalling tqdm-4.62.3:
      Successfully uninstalled tqdm-4.62.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datasets 1.12.1 requires tqdm>=4.62.1, but you have tqdm 4.49.0 which is incompatible.


In [29]:
from tqdm import tqdm

In [31]:
metric = load_metric("sacrebleu")

In [32]:
# Preprocessing

In [33]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

Downloading:   0%|          | 0.00/1.17k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/773k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.32M [00:00<?, ?B/s]

In [34]:
print(raw_datasets['test']['translation'][0]['sparql'])

select (COUNT(?sub) AS ?value ) [ ?sub wdt:P1582 wd:Q12024 ]


In [35]:
tokenizer.decode(tokenizer(raw_datasets['test']['translation'][0]['sparql'])['input_ids'])

'select (COUNT(?sub) AS?value ) [?sub wdt:P1582 wd:Q12024 ]</s>'

In [36]:
print(raw_datasets['test']['translation'][0]['en'],tokenizer(raw_datasets['test']['translation'][0]['en']))

When are Taxon products determined for Pinus? {'input_ids': [366, 33, 5287, 106, 494, 4187, 21, 8050, 302, 58, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


In [37]:
prefix = "translate English to Sparql: "

In [38]:
max_input_length = 0 
max_target_length = 0
for d in tqdm(raw_datasets['train']['translation']):
    len_en = len(d['en'])
    len_qry = len(d['sparql'])
    if len_en > max_input_length: max_input_length=len_en
    if len_qry > max_target_length: max_target_length=len_qry

100%|██████████| 38448/38448 [00:00<00:00, 729463.97it/s]


In [39]:
print(max_input_length, max_target_length)

248 224


In [40]:
source_lang = "en"
target_lang = "sparql"

def preprocess_function(examples):
    inputs = []
    targets= []
    for ex in examples["translation"]:
      inputs.append(prefix + ex[source_lang])
      targets.append(ex[target_lang])
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
      labels = tokenizer(targets, max_length=max_target_length, truncation=True)
      
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [41]:
tokenized_datasets = raw_datasets.map(preprocess_function, batched=True)

  0%|          | 0/39 [00:00<?, ?ba/s]

  0%|          | 0/10 [00:00<?, ?ba/s]

In [42]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['__index_level_0__', 'attention_mask', 'input_ids', 'labels', 'translation'],
        num_rows: 38448
    })
    test: Dataset({
        features: ['__index_level_0__', 'attention_mask', 'input_ids', 'labels', 'translation'],
        num_rows: 9612
    })
})

In [43]:
# Fine-tuning the model

In [44]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

Downloading:   0%|          | 0.00/231M [00:00<?, ?B/s]

In [45]:
batch_size = 8
args = Seq2SeqTrainingArguments(
    model_name,
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=1,
    predict_with_generate=True,
    fp16=True,
)

In [46]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [47]:
fake_preds = ["hello there", "general kenobi"]
fake_labels = [["hello there"], ["general kenobi"]]
metric.compute(predictions=fake_preds, references=fake_labels)

{'bp': 1.0,
 'counts': [4, 2, 0, 0],
 'precisions': [100.0, 100.0, 0.0, 0.0],
 'ref_len': 4,
 'score': 0.0,
 'sys_len': 4,
 'totals': [4, 2, 0, 0]}

In [48]:
# The last thing to define for our Seq2SeqTrainer is how to compute 
# the metrics from the predictions. We need to define 
# a function for this, which will just use the metric we loaded earlier, 
# and we have to do a bit of pre-processing to decode the predictions into texts:

import numpy as np

def postprocess_text(preds, labels):
    preds = [pred.replace('?',' ?').replace('.', ' .').strip() for pred in preds]
    labels = [[label.replace('?',' ?').replace('.', ' .').strip()] for label in labels]

    return preds, labels

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]}

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result

In [49]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

Using amp fp16 backend


In [56]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: __index_level_0__, translation.
***** Running training *****
  Num examples = 38448
  Num Epochs = 1
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 4806


Epoch,Training Loss,Validation Loss,Bleu,Gen Len
1,0.760700,0.644727,5.415600,19.000000


Saving model checkpoint to sparql-translator-2021-10-01-t5-small-fp16/checkpoint-500
Configuration saved in sparql-translator-2021-10-01-t5-small-fp16/checkpoint-500/config.json
Model weights saved in sparql-translator-2021-10-01-t5-small-fp16/checkpoint-500/pytorch_model.bin
tokenizer config file saved in sparql-translator-2021-10-01-t5-small-fp16/checkpoint-500/tokenizer_config.json
Special tokens file saved in sparql-translator-2021-10-01-t5-small-fp16/checkpoint-500/special_tokens_map.json
Saving model checkpoint to sparql-translator-2021-10-01-t5-small-fp16/checkpoint-1000
Configuration saved in sparql-translator-2021-10-01-t5-small-fp16/checkpoint-1000/config.json
Model weights saved in sparql-translator-2021-10-01-t5-small-fp16/checkpoint-1000/pytorch_model.bin
tokenizer config file saved in sparql-translator-2021-10-01-t5-small-fp16/checkpoint-1000/tokenizer_config.json
Special tokens file saved in sparql-translator-2021-10-01-t5-small-fp16/checkpoint-1000/special_tokens_map.js

TrainOutput(global_step=4806, training_loss=0.8635234316834994, metrics={'train_runtime': 2881.4302, 'train_samples_per_second': 13.343, 'train_steps_per_second': 1.668, 'total_flos': 1410855806435328.0, 'train_loss': 0.8635234316834994, 'epoch': 1.0})

In [57]:
trainer.save_model(model_path)

Saving model checkpoint to ./models/sparql-translator-2021-10-01-t5-small-fp16
Configuration saved in ./models/sparql-translator-2021-10-01-t5-small-fp16/config.json
Model weights saved in ./models/sparql-translator-2021-10-01-t5-small-fp16/pytorch_model.bin
tokenizer config file saved in ./models/sparql-translator-2021-10-01-t5-small-fp16/tokenizer_config.json
Special tokens file saved in ./models/sparql-translator-2021-10-01-t5-small-fp16/special_tokens_map.json


In [58]:
!ls -l --block-size=M {model_path}

total 233M
-rw-r--r-- 1 root root   1M Oct  1 23:32 config.json
-rw-r--r-- 1 root root 231M Oct  1 23:32 pytorch_model.bin
-rw-r--r-- 1 root root   1M Oct  1 23:32 special_tokens_map.json
-rw-r--r-- 1 root root   1M Oct  1 23:32 tokenizer_config.json
-rw-r--r-- 1 root root   2M Oct  1 23:32 tokenizer.json
-rw-r--r-- 1 root root   1M Oct  1 23:32 training_args.bin


In [61]:
!mkdir drive/MyDrive/models/{model_name}

In [62]:
!cp {model_path}/* drive/MyDrive/models/{model_name}/